In [1]:
#load "Paket.fsx"
Paket.Package ["Alea"; "NUnit"]

<null>

In [2]:
#load "packages/Alea/Alea.fsx"
#r "packages/NUnit/lib/net45/nunit.framework.dll"

In [3]:
open System
open Alea
open Alea.FSharp
open NUnit.Framework

let length = 1000000

let kernel =
    <@ fun (result:int[]) (arg1:int[]) (arg2:int[]) ->
        let start = blockIdx.x * blockDim.x + threadIdx.x
        let stride = gridDim.x * blockDim.x
        let mutable i = start
        while i < result.Length do
            result.[i] <- arg1.[i] + arg2.[i]
            i <- i + stride @>
    |> Compiler.makeKernel

In [9]:
let harg1 = Array.init length id
let harg2 = Array.init length id

let gpu = Gpu.Default
let lp = LaunchParam(16, 256)
let darg1 = gpu.Allocate(harg1)
let darg2 = gpu.Allocate(harg2)
let dresult = gpu.Allocate<int>(length)
gpu.Launch kernel lp dresult darg1 darg2
let actual = Gpu.CopyToHost(dresult)

Gpu.Free(darg1)
Gpu.Free(darg2)
Gpu.Free(dresult)

actual

[|0; 2; 4; 6; 8; 10; 12; 14; 16; 18; 20; 22; 24; 26; 28; 30; 32; 34; 36; 38; 40;
  42; 44; 46; 48; 50; 52; 54; 56; 58; 60; 62; 64; 66; 68; 70; 72; 74; 76; 78; 80;
  82; 84; 86; 88; 90; 92; 94; 96; 98; 100; 102; 104; 106; 108; 110; 112; 114;
  116; 118; 120; 122; 124; 126; 128; 130; 132; 134; 136; 138; 140; 142; 144; 146;
  148; 150; 152; 154; 156; 158; 160; 162; 164; 166; 168; 170; 172; 174; 176; 178;
  180; 182; 184; 186; 188; 190; 192; 194; 196; 198; ...|]

In [10]:
let expected = (harg1, harg2) ||> Array.map2 (fun a b -> a + b)
Assert.That(actual, Is.EqualTo(expected))

<null>